In [ ]:
!pip install kaggle

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 19% 5.00M/25.7M [00:00<00:00, 32.4MB/s]
100% 25.7M/25.7M [00:00<00:00, 117MB/s] 


In [ ]:
import sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
kaggel_dictinary = json.load(open('/content/kaggle.json'))

In [ ]:
kaggel_dictinary .keys()

dict_keys(['username', 'key'])

In [ ]:
os.environ['KAGGLE_USERNAME'] = kaggel_dictinary['username']
os.environ['KAGGLE_KEY'] = kaggel_dictinary['key']

In [ ]:
!ls


'archive (5).zip'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [ ]:
with ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip:
  zip.extractall()

In [ ]:
!ls

'archive (5).zip'    imdb-dataset-of-50k-movie-reviews.zip   sample_data
'IMDB Dataset.csv'   kaggle.json


loading the dataset

In [ ]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [ ]:
data.replace({"sentiment":{"positive":1,"negative":0}},inplace= True)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
print(train_data.shape)

(40000, 2)


#data prepocessing


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Assuming you have train_data and test_data defined somewhere
# train_data["review"] and test_data["review"] should be lists of strings

# Create a tokenizer and fit on the training data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])

# Convert text to sequences and pad sequences to a maximum length of 200
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)


In [ ]:
print(x_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [ ]:
y_train = train_data["sentiment"]
y_test = test_data["sentiment"]

In [ ]:
print(y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


##LSTM

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense
model = Sequential()
model.add(Embedding(5000, 128, input_length=200))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))


Epoch 1/5
1250/1250 [==============================] - 443s 353ms/step - loss: 0.4096 - accuracy: 0.8158 - val_loss: 0.4773 - val_accuracy: 0.7797
Epoch 2/5
1250/1250 [==============================] - 429s 343ms/step - loss: 0.3171 - accuracy: 0.8666 - val_loss: 0.2701 - val_accuracy: 0.8891
Epoch 3/5
1250/1250 [==============================] - 432s 346ms/step - loss: 0.2171 - accuracy: 0.9137 - val_loss: 0.2673 - val_accuracy: 0.8877
Epoch 4/5
1250/1250 [==============================] - 434s 347ms/step - loss: 0.1743 - accuracy: 0.9335 - val_loss: 0.2894 - val_accuracy: 0.8901
Epoch 5/5
1250/1250 [==============================] - 426s 341ms/step - loss: 0.1386 - accuracy: 0.9480 - val_loss: 0.2934 - val_accuracy: 0.8884


In [41]:
loss,accuracy =  model.evaluate(x_test,y_test)
print(loss,accuracy)

313/313 [==============================] - 53s 169ms/step - loss: 0.2934 - accuracy: 0.8884
0.2933538556098938 0.8884000182151794


In [42]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [43]:
# example usage
new_review = "This movie was not that good"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 1s 618ms/step
The sentiment of the review is: negative
